In [ ]:
import re

j = 0

notyears = ['Argonautica','Dicaearchus','Editions','Epigram','Eutrop','Gennadius','Hephaestion','Liberalis','Lycophron','Petronius','Theogony','appended','chapter','epigrams','wrote']
notyears.append('Apollo')
notyears.append('Aphrodite')
notyears.append('Aristophanes')
notyears.append('Artemis')
notyears.append('Avienus')
notyears.append('Chius')
notyears.append('Demeter')
notyears.append('Festus')
notyears.append('Jornandes')
notyears.append('Obsequens')
notyears.append('Orelli')
notyears.append('Orpheus')
notyears.append('Periegetes')
notyears.append('Phrygius')
notyears.append('Scymnus')
notyears.append('Thesmophoriazusae')

rnums = {}
rnums['i'] = 1
rnums['ii'] = 2
rnums['iii'] = 3
rnums['iv'] = 4
rnums['v'] = 5
rnums['vi'] = 6
rnums['vii'] = 7
rnums['viii'] = 8
rnums['ix'] = 9
rnums['x'] = 10
rnums['xi'] = 11
rnums['xii'] = 12
rnums['xiii'] = 13
rnums['xiv'] = 14
rnums['xv'] = 15
rnums['xvi'] = 16
rnums['xvii'] = 17
rnums['xviii'] = 18
rnums['xix'] = 19
rnums['xx'] = 20
rnums['xxi'] = 21
rnums['xxii'] = 22
rnums['xxiii'] = 23
rnums['xxiv'] = 24
rnums['xxv'] = 25
rnums['xxvi'] = 26
rnums['xxvii'] = 27
rnums['xxviii'] = 28
rnums['xxix'] = 29
rnums['xxx'] = 30
rnums['xxxi'] = 31
rnums['xxxii'] = 32
rnums['xxxiii'] = 33
rnums['xxxiv'] = 34
rnums['xxxv'] = 35
rnums['xxxvi'] = 36
rnums['xxxvii'] = 37
rnums['xxxviii'] = 38
rnums['xxxix'] = 39
rnums['xl'] = 40
rnums['xli'] = 41
rnums['xlii'] = 42
rnums['xliii'] = 43
rnums['xliv'] = 44
rnums['xlv'] = 45
rnums['xlvi'] = 46
rnums['xlvii'] = 47
rnums['xlviii'] = 48
rnums['xlix'] = 49
rnums['l'] = 50


countwords = []
f2 = open('../002/count-words.txt')
for l in f2:
    l = re.sub('\n','',l)
    countwords.append(l)
f2.close()

def fract2dec(s):
    s = re.sub(',','',s)
    m = re.search('([0-9]+)\s+([0-9]+)/([0-9]+)',s)
    if(m):
        basen = int(m[1])
        numerator = int(m[2])
        denom = int(m[3])
        rval = basen + numerator/denom
        print('fractres',s,rval)
        s = str(rval)
        
    return(s)


f = open("viaf88890045.003.perseus-eng1.xml")
outf = open('smithbio-work1.xml','w')

entrylens = {}

idlist = {}
inheader = 1
needspace = 0
curvolume = ''
curentryid = ''
firstpara = 0
baseid = ''
for l in f:
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue
        
    m = re.search('<milestone unit="volume" n="([123])"/>',l)
    if(m):
        curvolume = m[1]
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        firstpara = 1
        curentryid = m[1]
        baseid = re.sub('-[0-9a-z]+$','',curentryid)
        if( baseid in idlist):
            idlist[baseid] = idlist[baseid] + 1
        else:
            idlist[baseid] = 1
        newid = baseid + '-' + str(idlist[baseid])
        if(not newid == curentryid):
            #print('changeid',curentryid,newid)
            l = re.sub('id="'+curentryid+'"','id="'+newid+'"',l)
            curentryid = newid

        entrylens[curentryid] = 0
    
    
    if( re.search('<p>',l)):
        m = re.search('<p>([0-9]+)\.',l)
        if(m and firstpara and baseid and not str(idlist[baseid]) == m[1]):
            print('badn',curentryid,baseid,idlist[baseid],l)
        firstpara = 0
    if(curentryid in entrylens):
        entrylens[curentryid] = entrylens[curentryid] + len(re.sub('<[^<]+>','',l).split())


    l = re.sub('\s*\n',' ',l)
    l = re.sub('^\s+','',l)
    l = re.sub('([,;])</hi>','</hi>\g<1>',l)
    if(1 and re.search('^\s*(<div|<by|<p>|</div>)',l)):
        l = re.sub('^\s*(<item|<div|<by|<p>|</div>)','\n\n\g<1>',l)
    if(re.search('^\s*(<head|<milestone)',l)):
        l = re.sub('^\s*(<head|<milestone)','\n\g<1>',l)
    print(l,end='',file=outf)

f.close()
outf.close()

singnameindex = {}
for foo in entrylens:
    workid = re.sub('-bio.*','',foo)
    for tmpkey in workid.split('-'):
        if(len(tmpkey)<3):
            continue
        if(not tmpkey in singnameindex):
            singnameindex[tmpkey] = {}
        singnameindex[tmpkey][foo] = entrylens[foo]

f = open('smithbio-work1.xml')
outf = open('smithbio-work.xml','w')

entryheads = {}
entrydates = {}
entrytitles = {}
sentences = 0
inheader = 1
curpage = ''
curvolume = ''
curentryid = ''

shortnames = ['Cato','Hera','Jew','Jews','Livy','Milo','Num','Otho','Ovid','Piso','Pius','Way','Ways','West','Zeno','Zeus']
shortpat = '('+ '|'.join(shortnames) + ')'

newdates = 0
newdates1 = 0
newdates2 = 0
newdates3 = 0

centval = {}
centval["15th B.C."] = "-1500-1401"
centval["twelfth B.C."] = "-1200-1101"
centval["tenth B.C."] = "-1000-901"
centval["eleventh B.C."] = "-1100-1001"
centval["ninth B.C."] = "-900-801"
centval["8th B.C."] = "-800-701"
centval["eighth B.C."] = "-800-701"
centval["seventh B.C."] = "-700-601"
centval["7th B.C."] = "-700-601"
centval["sixth B.C."] = "-600-501"
centval["6th B.C."] = "-600-501"
centval["5th B.C."] = "-500-401"
centval["fifth B.C."] = "-500-401"
centval["fourth B.C."] = "-100-101"
centval["4th B.C."] = "-400-301"
centval["3rd B.C."] = "-300-201"
centval["third B.C."] = "-300-201"
centval["second B.C."] = "-200-101"
centval["1st B.C."] = "-100-001"
centval["first B.C."] = "-100-001"
centval["a B.C."] = "-100-001"
centval["the B.C."] = "-100-001"
centval["last B.C."] = "-100-001"
centval["first A.D."] = "001-100"
centval["2nd A.D."] = "101-200"
centval["second A.D."] = "101-200"
centval["3rd A.D."] = "201-300"
centval["third A.D."] = "201-300"
centval["4th A.D."] = "301-400"
centval["fourth A.D."] = "301-400"
centval["fifth A.D."] = "401-500"
centval["5th A.D."] = "401-500"
centval["sixth A.D."] = "501-600"
centval["6th A.D."] = "501-600"
centval["seventh A.D."] = "601-700"
centval["7th A.D."] = "601-700"
centval["eighth A.D."] = "701-800"
centval["8th A.D."] = "701-800"
centval["ninth A.D."] = "801-900"

centval["9th A.D."] = "801-900"
centval["10th A.D."] = "901-1000"
centval["tenth A.D."] = "901-1000"
centval["eleventh A.D."] = "1001-1100"
centval["12th A.D."] = "1101-1200"
centval["twelfth A.D."] = "1101-1200"
centval["thirteenth A.D."] = "1201-1300"
centval["13th A.D."] = "1201-1300"
centval["fourteenth A.D."] = "1301-1400"
centval["14th A.D."] = "1301-1400"
centval["fifteenth A.D."] = "1401-1500"
centval["15th A.D."] = "1401-1500"
centval["sixteenth A.D."] = "1501-1600"
centval["16th A.D."] = "1501-1600"

goodxrefs = 0
ambxrefs = 0
xrefs = 0
for l in f:
    l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)</hi>','<title>\g<1></title>',l)
    m = re.search('<milestone unit="volume" n="([0-9]+)"',l)
    l,hits = re.subn('(\s+|\()(A\.\s*D\.)\s*([0-9]+)','\g<1><date when-custom="\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('(\s+|\()(B\.\s*C\.)\s*([0-9]+)','\g<1><date when-custom="-\g<3>">\g<2> \g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('([0-9]+)([,\s]*)(B\.\s*C\.)','<date when-custom="-\g<1>">\g<1>\g<2>\g<3></date>',l)
    newdates1 = newdates1 + hits
    l,hits = re.subn('([0-9]+)([,\s]*)(A\.\s*D\.)','<date when-custom="\g<1>">\g<1>\g<2>\g<3></date>',l)
    newdates1 = newdates1 + hits
    if(m):
        curvolume = m[1]
        
    m = re.search('<pb n="([0-9ivx]+)"',l)
    if(m):
        curpage = m[1]
        if( curvolume == '1'):
        
            if(re.search('^[0-9]+$',curpage)):
                l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=' + str(18+int(curpage))+'"',l)
            elif(curpage in rnums):
                l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=' + str(6+int(rnums[curpage]))+'"',l)
    
        if( curvolume == '2'):
            l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044011634888%25seq=' + str(26+int(curpage))+'"',l)
        if( curvolume == '3'):
            l = re.sub('(<pb[^>/]+)','\g<1> source="https://babel.hathitrust.org/cgi/pt?id=nnc1.cr00106801%25seq=' + str(8+int(curpage))+'"',l)
        
    if(re.search('<head',l)):
        curhead = re.sub('<[^>]+>','',l)
        curhead = re.sub('\n','',curhead)
        if( not curentryid in entryheads):
            entryheads[curentryid] = curhead
        
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        #entrylens[curentryid] = 0
        entrydates[curentryid] = []
        entrytitles[curentryid] = []
        
    if(inheader):
        if(re.search('</teiHeader>',l)):
            inheader = 0
        print(l,end='',file=outf)
        continue

    i = 0 
    matchs = '<date when-custom="([0-9]+|\-[0-9]+)">([^<]+</date>)(\s+and\s+|\s+or\s+|\s+to\s+|\s+till\s+|,\s+)([0-9]+|\-[0-9]+)'
    m = re.search(matchs,l)
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        newdates = newdates + 1
        date1 = m[1]
        linkword = m[3]
        date2 = m[4]
        if(re.search('^\-',date1)):
            date2 = '-' + date2
            #print('date2 now',date1,date2,l)
        i = i + 1
        if(i> 100):
            print('outofcontrol1',l)
            break
        if(0 and int(date2) < int(date1)):
            print('newdate',curentryid,curvolume,curpage,date1,linkword,date2,l,end='')
        l = re.sub(matchs,'<date when-custom="\g<1>">\g<2>\g<3><date when-custom="\g<4>">\g<4></date>',l,1)


    l,hits = re.subn('(</date>)\s*-\s*([0-9]+)','\g<1>-<date when-custom="\g<2>">\g<2></date>',l)
    newdates2 = newdates2 + hits
    
    workl = l
    matchs = '<date when-custom="([0-9\-]+)">([^<]+)</date>-<date when-custom="([0-9\-]+)">'
    while(re.search(matchs,workl)):
        m = re.search(matchs,workl)
        date1 = m[1]
        date2 = m[3]
        if(not len(date1) == len(date2) and int(date1) > int(date2)):
            datemask = re.sub('(.)','.',date2) + '$'
            half1 = re.sub(datemask,'',date1)
            newdate2 = half1 + date2
            if(1 or int(newdate2) < int(date1)):
                print('unmatched',date1,date2,newdate2)
            l = re.sub('when-custom="' + date2+ '"','when-custom="' + newdate2 + '"',l)
        workl = re.sub(matchs,'',workl,1)


        
    
    workl = l
    while(re.search('<title[^>]*>([^<]+)',workl)):
        m = re.search('<title[^>]*>([^<]+)',workl)
        if(curentryid in entrytitles):
            entrytitles[curentryid].append(m[1])
        workl = re.sub('<title[^>]*>([^<]+)','',workl,1)
        
    workl = l
        
    l = re.sub('(<hi rend="smallcaps">[^<]+)</hi>\s+<hi rend="smallcaps">','\g<1> ',l)
    if(re.search('<p>',l)):
        l = re.sub('([A-Z][a-z][a-z]\.)','<abbr>\g<1></abbr>',l)
        l = re.sub('<abbr>'+shortpat+'\.</abbr>','\g<1>',l)
        while(re.search('(="[^"]*|<bibl[^>]*>[^<]*|<title[^>]*>[^<]*)<[/]*abbr>',l)):
            l = re.sub('(="[^"]*|<bibl[^>]*>[^<]*|<title[^>]*>[^<]*)<[/]*abbr>','\g<1>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+[A-Za-z]+[,\s]|[A-Z][a-z]*\s+<[a-zA-Z]+)','\g<1>\n<s n="r01"/>\g<2>',l)
        l,hits = re.subn('(\)\.|\.\)|\.</hi>\)|\.</foreign>\))\s+([A-Z])','\g<1>\n<s n="r2"/>\g<2>',l)
        l,hits = re.subn('(</date>\.)\s+([A-Z])','\g<1>\n<s n="r3"/>\g<2>',l)
        l,hits = re.subn('(\]\.|\.\)])\s+([A-Z])','\g<1>\n<s n="r4"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*,\s+[A-Za-z]+[,\s])','\g<1>\n<s n="r5"/>\g<2>',l)
        l,hits = re.subn('([a-z][a-z]+[\.!?"])\s+([A-Z][a-z]*\s+\()','\g<1>\n<s n="r6"/>\g<2>',l)
        l,hits = re.subn('(\.</hi>\])\s+([A-Z])','\g<1>\n<s n="r7"/>\g<2>',l)
        l,hits = re.subn('(\.)\s+(In\s+<date)','\g<1>\n<s n="r8"/>\g<2>',l)
        l,hits = re.subn('(\.</foreign>)\s+([A-Z])','\g<1>\n<s n="r9"/>\g<2>',l)
        l,hits = re.subn('(</ref>\]|</bibl>\))\s+([A-Z])','\g<1>\n<s n="r11"/>\g<2>',l)
        l,hits = re.subn('(\.</hi>)\s+([A-Z][a-z]+\s)','\g<1>\n<s n="r12"/>\g<2>',l)

    
    matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.|after\s+Christ|before Christ)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        if(m[3] == 'after Christ'):
            era = 'A.D.'
        elif(m[3] == 'before Christ'):
            era = 'B.C.'
        else:
            era = m[3]
        curkey = m[1] + ' ' + era
        curkey = re.sub('\.\s+','.',curkey)
        if(not curkey in centval):
            print('missingcentval',curkey,l)
            curval = 'noval'
        else:
            curval = centval[curkey]
        l = re.sub(matchs,'<date when-custom="'+curval+'">\g<1> \g<2> <z/>\g<3></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat = '([0-9]+)([a-z]+\s+Olympiad)'
    while(re.search(olymppat,l)):
        m = re.search(olymppat,l)
        ol = int(m[1])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat,'<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
    
    olymppat2 = '(\(Ol\.\s+|Ol\.\s+)([0-9]+)'
    while(re.search(olymppat2,l)):
        m = re.search(olymppat2,l)
        ol = int(m[2])
        startyear = -776 + (4*(ol-1))
        endyear = startyear + 3
        yearrange = str(startyear) + '-' + str(endyear)
        l = re.sub(olymppat2,'(<date when-custom="'+yearrange+'">\g<1><z/>\g<2></date>',l,1)
    l = re.sub('<z/>','',l)
 
    i = 0
    workl = l
    while(re.search('<date when-custom="([\-0-9]+)"',workl)):
        m = re.search('<date when-custom="([\-0-9]+)"',workl)
        i = i + 1
        if(i> 100):
            print('outofcontrol12',l)
            break
        entrydates[curentryid].append(m[1])
        workl = re.sub('<date when-custom="([\-0-9]+)"','',workl,1)

    l,hits = re.subn('([0-9]+)(\s+to\s+<date when-custom="-)','<date when-custom="-\g<1>">\g<1></date>\g<2>',l)
    newdates3 = newdates3 + hits

    if( re.search('<p>\[<hi rend="smallcaps">[^<]+</hi>\.\]</p>',l)):
            print(l,end='',file=outf)
            continue
            

    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)(,\s+No\.\s+)([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        curnum = m[5]
        xrefkey = re.sub('[,\s]+','-',curname.lower()) + '-bio-' + curnum
        xrefs = xrefs + 1
        if(xrefkey in entrylens):
            goodxrefs = goodxrefs + 1
            res = 'hit'
            l = re.sub(matchs,'<ref target="'+xrefkey+'">\g<2></z>\g<4>\g<5></ref>',l,1)
        else:
            res = 'fail'
            print('xref',res,curentryid,curname,curnum,xrefkey,xrefs,goodxrefs)
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2>\g<3>\g<4>\g<5>',l,1)
    l = re.sub('</z>','',l)       
    matchs = '(<hi rend="smallcaps">)([^<]+)(</hi>)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        curname = m[2]
        xrefkey1 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-1'
        xrefkey2 = re.sub('[,\.\s]+','-',curname.lower()) + '-bio-2'
        xrefkey1 = re.sub('[\-]+','-',xrefkey1)
        xrefkey2 = re.sub('[\-]+','-',xrefkey2)
        xrefs = xrefs + 1
        if(xrefkey1 in entrylens):
            if(xrefkey2 in entrylens):
                res ='amb'
                ambxrefs = ambxrefs + 1

            else:
                res = 'hit'
                goodxrefs = goodxrefs + 1
                
            l = re.sub(matchs,'<ref target="'+xrefkey1+'">\g<2></ref>',l,1)
        else:
            res = 'fail'
            l = re.sub(matchs,'<hix rend="smallcaps">\g<2></hi>',l,1)
        if(0 and not res == 'hit'):
            print('xref2',res,curentryid,curname,xrefkey1,xrefs,goodxrefs,ambxrefs)


    l = re.sub('<hix ','<hi ',l)
    
    l = re.sub('(<bibl n="[^"]+ p)\.','\g<1>age',l)
    l = re.sub('(<bibl n="[^"]+ p)p\.','\g<1>ages',l)
    matchs = '\\b(p\.|pp\.)\s+([0-9][0-9\-]*[a-f]*)'
    l = re.sub(matchs,'<ref type="page" n="\g<2>">\g<1> \g<2></ref>',l)
    l = re.sub('(</ref>)([,\s]+seq\.)','\g<2>\g<1>',l)

    l = re.sub('(Orelli|Orell\.)([,\s]+|[,\s]+[Nn]o[s]*\.\s+)([0-9]+)','<bibl n="Orelli Inscr \g<3>">\g<1>\g<2>\g<3></bibl>',l)
    
    matchs = '(Orell[\.i][,\s+]+)<title>([^<]+)</title>(\s+|[,\s]+[Nn]o[s]*\.\s+)([0-9][0-9\-]*)'
    l = re.sub(matchs,'<bibl n="Orelli Inscr \g<4>">\g<1>\g<2>\g<3>\g<4></bibl>',l)
    
    matchs = '(<bibl n="Orelli Inscr\s+)([^>]+>[^<]+</bibl>)(,\s+)([0-9][0-9\-]*)'
    while(re.search(matchs,l)):
        l = re.sub(matchs,'\g<1>\g<2>\g<3>\g<1>\g<4>">\g<4></bibl>',l,1)
        

    matchs = '(vol\.\s+)([ivx]+)'
    
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        volnum = rnums[m[2]]
        l = re.sub(matchs,'<ref type="volume" n="'+str(volnum)+'">\g<1></z>\g<2></ref>',l)
    l = re.sub('</z>','',l)
    
    matchs = '(<ref type="page"[^>]+>[^<]+</ref>[,\s]+)([0-9][0-9\-]*[a-f]*)'
    while(re.search(matchs,l)):
        l = re.sub(matchs,'\g<1><ref type="page" n="\g<2>">\g<2></ref>',l)
        l = re.sub('(</ref>)([,\s]+seq\.)','\g<2>\g<1>',l)
        
    l = re.sub('(</ref>[,\s]+No[s]\.)\s+([0-9]+)','\g<1><ref type="num" n="\g<2>">\g<1> \g<2>',l)

    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+([A-Z][a-z]+|geog[\.a-z]+|heroic|heav[ily]+\-armed|iambic-trimeter|iambic|mercenary|new|of\s+the[a-z]*|sq\.|square|select)\s+([a-z\-]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = m[2]
        word2 = m[3]
        if(word2 in countwords):
            l = re.sub(matchs,'<measure quantity="'+number+'" n="\g<2> \g<3>">\g<1> \g<2> \g<3></measure>',l,1)
        l = re.sub(matchs,'\g<1></z> \g<2> \g<3>',l)
    
    l = re.sub('</z>','',l)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+(M\.\s*P\.)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = re.sub(' ','',m[2])
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+)\s+(vols\.|vol\.)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = re.sub(',','',m[1])
        word1 = re.sub(' ','',m[2])

        l = re.sub(matchs,'<measure quantity="'+number+'" n="volumes">\g<1></z> \g<2></measure>',l,1)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+|[0-9]+\s+[0-9]+/[0-9]+)\s+([a-z\-]+|[A-Z][a-z]+s\\b)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        number = fract2dec(m[1])
        word1 = m[2]

        if(word1 in countwords or re.search('[A-Z].+s$',word1)):
            l = re.sub(matchs,'<measure quantity="'+number+'" n="\g<2>">\g<1> \g<2></measure>',l,1)
        l = re.sub(matchs,'\g<1></z> \g<2>',l,1)
    l = re.sub('</z>','',l)
    
    matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+)\s*l\.\s*([0-9]+)\s*s\.\s+([0-9]+)\s*d\.'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
        pounds = re.sub(',','',m[1])
        l = re.sub(matchs,'<measure n="pounds-shillings-pence">\g<1>l.</z> \g<2>s. \g<3>d.</measure>',l)
    l = re.sub('</z>','',l)

    matchs = '\\b([Ii]n\s+)([0-9]+\.[0-9]+\.[0-9]+|[0-9]+\.[0-9]+)'

    l = re.sub(matchs,'\g<1><ref>\g<2></ref>',l)
    
    l = re.sub('(4to\.|4to)','<measure n="quarto">\g<1></measure>',l)
    l = re.sub('(8vo\.|8vo)','<measure n="octavo">\g<1></measure>',l)
    
    
    matchs = '([0-9]+)°\s+([0-9]+)′\s+([0-9]+)″'
    while(re.search(matchs,l)):
          m = re.search(matchs,l)
          print(curentryid,m[1],m[2],m[3])
          l = re.sub(matchs,'<measure type="degree" quantity="\g<1>">\g<1></z>°</measure> <measure type="minute" quantity="\g<2>">\g<2></z>′</measure> <measure type="second" quantity="\g<3>">\g<3></z>″</measure>',l)
          
    l = re.sub('</z>','',l)      
          
    matchs = '([0-9]+)(\s+or\s+|\s+and\s+|\s+to\s+)<measure quantity="([^"]+)" n="([^"]+)">'

    l = re.sub(matchs,'<measure quantity="\g<1>" n="\g<4>">\g<1></measure>\g<2><measure quantity="\g<3>" n="\g<4>">',l)

    
    m = re.search('<bibl n="([A-Z][a-z]+)',l)
    if(m):
        if(not m[1] in lastbib):
            lastbib.append( m[1])
        
    m = re.search('<date when-custom="([^"]+)"',l)
    if(m):
            lastdate = m[1]
            

    
    matchs = '\\b([Ii]n\s+)([0-9]+\.[0-9]+\.[0-9]+|[0-9]+\.[0-9]+)'

    l = re.sub(matchs,'\g<1><ref>\g<2></ref>',l)
    

    
    m = re.search('entry" xml:id="([^"]+)',l)
    if(m):
        curentryid = m[1]
        lastdate = 'nolast'
        lastbib = []
    
    matchs = '\\b([a-zA-Z]+)\s+([a-zA-Z]+)\s+\(([0-9]+)\)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
    
        if(m and int(m[3]) > 9 and int(m[3]) < 1000 and not m[2] in notyears):
            j = j + 1
            if(re.search('^-',lastdate) or 'Liv' in lastbib or 'Cic' in lastbib or 'Thuc' in lastbib or 'Dionys' in lastbib):
                newdate = '-' + m[3]
            else:
                newdate = m[3]
            print(j,'paren',curentryid,lastdate,'Cic' in lastbib,lastbib,m[1],m[2],m[3],newdate)
            l = re.sub(matchs,'\g<1> \g<2> (<date when-custom="'+newdate+'">\g<3></date>)',l,1)
        else:
            l = re.sub(matchs,'\g<1> \g<2> (<ref>\g<3></ref>)',l,1)
    
    matchs = '\\b([Ii]n)\s([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)            
        newdate = m[2]
        if(re.search('^-',lastdate) or  'Plb' in lastbib or 'Liv' in lastbib or 'Cic' in lastbib):
            if(int(m[2])< 900):
                newdate = '-' + m[2]
            else:
                newdate = m[2]
        else:
            newdate = m[2]
        if(len(newdate) == 1):
            shows = l
        else:
            shows = ''
        i = i + 1
        if(lastdate == 'nolast' and not lastbib == 'nobib' and int(newdate)  < 1000):
            print(i,curentryid,'newdate',lastdate,lastbib,newdate,shows)
        l =re.sub(matchs,'\g<1> <date when-custom="'+newdate+'">\g<2></date>',l,1)
    
    
    
    print(l,end='',file=outf)
outf.close()

print('done')


badn p-consentius-bio-1.2-1 p-consentius-bio-1.2 1        <p>2. <hi rend="smallcaps">CONSENTIUS</hi>, who rose to high honour under Valentinian III., by whom he was named Comes Palatii and despatched upon an important mission to Theodosius. He also had a son, namely</p>

badn p-consentius-bio-1.3-1 p-consentius-bio-1.3 1        <p>3. <hi rend="smallcaps">CONSENTIUS</hi>, who devoted himself to literary leisure and the enjoyments of a rural life, and is celebrated as well as his grandfather by Sidonius.</p>

badn gratianus-augustus-bio-1 gratianus-augustus-bio 1      <p>2. <hi rend="smallcaps">GRATIANUS

badn gratianus-bio-2 gratianus-bio 2      <p>3. Theodosius the Great appears to have had a son Gratianus by his second wife Galla [<hi rend="smallcaps">GALLA</hi>, No. 2]: the child died before his father. (Ambros. <hi rend="ital">Epistol.</hi> 17, <hi rend="ital">De Obitu Theodos.</hi> 40, ed. Benedictin., with the editor's notes in both places.)</p>

badn gratianus-bio-3 gratianus-bio

badn herodotus-bio-10 herodotus-bio 10      <p>2. The son of Arieus, a native either of Tarsus or Philadelphia, who probably belonged to the sect of the Empirici. He was a pupil of Menodotus, and tutor to Sextus Empiricus, and lived therefore in the former half of the second century after Christ. (Suidas, <hi rend="ital">s. v.</hi>

badn herodotus-bio-11 herodotus-bio 11      <p>3. The physician mentioned by Galen (<hi rend="ital">De Bon. et Prav. Aliment. Succ.</hi>

badn heron-bio-4 heron-bio 4      <p>2. The teacher of Proclus, of whom nothing more is known. Fabricius (<hi rend="ital">Bibl. Graec.</hi> vol iv. p. 239) takes this to be the Heron who is mentioned by Eutocius as the commentator on the arithmetic of Nicomachus.</p>

badn heron-bio-5 heron-bio 5      <p>3. The younger, so called because we have not even an adjective of place to distinguish him from Heron of Alexandria, is supposed to have lived under Heraclius (A. D. 610-641). In his own work on Geodesy (a term used in t

badn hortensius-bio-7 hortensius-bio 7      <p>8. <persName xml:lang="la"><forename full="yes">Q.</forename> <surname full="yes">Hortensius</surname> <addName full="yes">Hortalus</addName></persName>, Q. F. L. N., son of the great orator, by Lutatia. His education was probably little cared for, for Cicero attributes his profligacy to the corrupting influence of one Salvius, a freedman (<hi rend="ital">ad Att.</hi> 10.18). On his return from his province, in <date when-custom="-50">B. C. 50</date>, Cicero found him at Laodicea, living with gladiators and other low company (<hi rend="ital">ad Att.</hi> 6.3). From the expressions in the same place, it appears that his father had cast him off; and we learn from other authority that he purposed to make his nephew, Messalla, his heir, to the exclusion of this son. (Val. Malx. 5.9.2.) However, he came in for part, at least, of his father's property; for we find Cicero inquiring what he was likely to offer for sale to satisfy his creditors (<h

badn joannes-bio-90 joannes-bio 90      <p>87. <hi rend="smallcaps">MAXENTIUS.</hi> [<hi rend="smallcaps">MAXENTIUS.</hi>]</p>

badn joannes-bio-91 joannes-bio 91      <p>88. <hi rend="smallcaps">MONACHUS.</hi> [See No. 106.]</p>

badn joannes-bio-92 joannes-bio 92      <p>89. <hi rend="smallcaps">MOCHUS.</hi> [<hi rend="smallcaps">MOSCHUS.</hi>]</p>

badn joannes-bio-93 joannes-bio 93      <p>90. <hi rend="smallcaps">NEPOS.</hi> [See No. 71.]</p>

badn joannes-bio-94 joannes-bio 94      <p>91. <hi rend="smallcaps">NESTEUTA.</hi> [See No. 28.]</p>

badn joannes-bio-95 joannes-bio 95      <p>92. (1). Joannes, archbishop of Nice before the 11th century.</p>

badn joannes-bio-96 joannes-bio 96      <p>93. (2). [See No. 21.]</p>

badn joannes-bio-97 joannes-bio 97      <p>94. Joannes, presbyter of the church of Nicomedeia in Bithynia, in the time of Constantine the Great.</p>

badn joannes-bio-98 joannes-bio 98      <p>95. <hi rend="smallcaps">OBEDIENTIAE

badn joannes-bio-99 joannes-bio 9

badn justinus-bio-5 justinus-bio 5      <p>2. In the <title>Acta S. Anastasii Persae Martyris,</title> of which two Latin versions are given in <hi rend="ital">the Acta Sanctorum, Januar.</hi> vol. ii. p. 426, &amp;c., mention is made of Justin, who was abbot of the monastery of St. Anastasius, about four miles distant from Jerusalem, about <date when-custom="620">A. D. 620</date>.</p>

badn justinus-bio-6 justinus-bio 6      <p>3. bishop of one of the sees in that island in the latter part of the fifth century. He was present at a council held at Rome <date when-custom="483">A. D. 483</date> or 484, under Pope Felix III., in which Petrus Fullo <foreign xml:lang="grc">Γναφεὺς</foreign>), or Peter the Fuller, patriarch of Antioch, was condemned as a heretic, for having added to the " trisagion" the heretical words " who suffered for us." Several bishops, among whom <pb n="687"/> was Justin, desirous of recalling Peter from his errors, addressed letters to him.</p>

badn labienus-bio-2 l

...

badn theopompus-bio-3 theopompus-bio 3      <p>2. the author of a work on earthquakes, quoted by Phlegon (<hi rend="ital">de Reb. Mirab. 19</hi>)</p>

badn theopompus-bio-4 theopompus-bio 4      <p>3. an epic poet, whose book en-titled <title xml:lang="grc">ἁρμάτιον</title> is quoted by Athenaeus (iv. p. 183b.; comp. Fulgent. <hi rend="ital">Mythol.</hi> p. 36; Schol. <hi rend="ital">ad Apollon. Rhod.</hi> 4.57; Vossius, <hi rend="ital">de Hist. Graec.</hi> p. 40, ed. Westermann, in whose note two or three other persons of the name are mentioned). </p>

badn thessalus-bio-7 thessalus-bio 7      <p>1. A son of Hippocrates, brother of Dracon I., and father of Gorgias, <note anchored="true" place="margin">* So it is stated by Meibomius (<hi rend="ital">Comment. in Hippocr. " Jusjur."</hi> p. 7) and other modern authors, but the Writer has hitherto been unable to find any ancient author who says that Thessalus had a son named Gorgias.</note> Hippocrates III. (Jo. Tzetzes, <hi rend="ital">C

badn venno-bio-1 venno-bio 1      <p>2. <persName xml:lang="la"><forename full="yes">L.</forename> <surname full="yes">Plautius</surname> <addName full="yes">Venno</addName></persName>, consul <date when-custom="-330">B. C. 330</date> with L. Papirius Crassus, carried on war with his colleague against the Privernates and Fundani. (<bibl n="Liv. 8.19">Liv. 8.19</bibl>; <bibl n="Diod. 17.82">Diod. 17.82</bibl>.) [<hi rend="smallcaps">VACCUS</hi>.]</p>

badn venno-bio-2 venno-bio 2      <p>3. <persName xml:lang="la"><forename full="yes">L.</forename> <surname full="yes">Plautius</surname> <addName full="yes">Venno</addName></persName>, L. F. L. N., consul <date when-custom="-318">B. C. 318</date> with M. Foslius Flaccinator, received hostages from the Teanenses and Canusini in Apulia. (Fasti Capit.; <bibl n="Liv. 9.20">Liv. 9.20</bibl>; <bibl n="Diod. 19.2">Diod.

badn vennonius-bio-3 vennonius-bio 3      <p>1. <persName xml:lang="la"><forename full="yes">SEX.</forename> <surname full="ye

badn zoilus-bio-10 zoilus-bio 10      <p>2. A sculptor of Corinthian vases, in the household of Agrippa, according to Raoul-Rochette's interpretation of the inscription, ZOILI . CORINTHIAR . AGRIPP. The matter is, however, doubtful. (R. Rochette, <hi rend="ital">Lettre à M. Schorn,</hi> p. 430,

badn zopyrus-bio-6 zopyrus-bio 6      <p>2. a rhetorician, was a contemporary of Timon. (<bibl n="Quint. Inst. 3.6.3">Quint. Inst. 3.6.3</bibl>; <bibl n="D. L. 9.114">D. L. 9.114</bibl>.)</p>

badn zopyrus-bio-7 zopyrus-bio 7      <p>3. an historian (Plut. <hi rend="ital">Parall. Min.</hi> c. 36), was probably the author of <foreign xml:lang="grc">Μιλήτου κτίσις</foreign>, the fourth book of which is cited by the Scholiast on Homer (<bibl n="Hom. Il. 10.274">Hom. Il. 10.274</bibl>). He is perhaps the same person as the Zopyrus mentioned by Marcellinus (<hi rend="ital">Vit. Thuc.</hi> § 32). Stobaeus quotes two verses from Zopyrus (<hi rend="ital">Floril.</hi>

badn zopyrus-bio-9 zopyrus-bio 9  

unmatched 460 61 461
unmatched 393 4 394
unmatched 413 14 414
unmatched 449 50 450
unmatched 460 61 461
unmatched 530 33 533
80 paren p-falcidius-bio-1 -40 False [] Manilian law 19 -19
unmatched 196 7 197
unmatched 357 61 361
unmatched 382 3 383
81 paren florentius-bio-1 361 False ['Amm'] same year 361 361
xref fail florus-gessius-bio-1 AGRIPPA HERODES 2 agrippa-herodes-bio-2 3102 886
unmatched 144 5 145
unmatched 146 7 147
unmatched 149 50 150
unmatched 163 4 164
unmatched 161 2 162
unmatched 161 2 162
unmatched 167 8 168
82 paren galla-bio-3 408 False [] next year 417 417
1 genesius-josephus-bio-1 newdate nolast [] 886 
unmatched 1438 39 1439
xref fail georgius-literary-bio-38 GEORGIUS EPARCHUS 23 georgius-eparchus-bio-23 3366 948
xref fail georgius-literary-bio-44 GEORGIUS 36 georgius-bio-36 3381 952
xref fail georgius-literary-bio-45 GENNADIUS OF CONSTANTINOPLE 2 gennadius-of-constantinople-bio-2 3382 952
xref fail granius-bio-1 CRASSUS 23 crassus-bio-23 3451 969
xref fail gregoriu

xref fail metellus-bio-22 CRASSUS 29 crassus-bio-29 5823 1708
1 methodius-bio-1 newdate nolast [] 853 
2 methodius-bio-1 newdate nolast [] 861 
3 methodius-bio-1 newdate nolast [] 878 
147 paren methodius-bio-2 nolast False [] his accession 842 842
1 methodius-bio-2 newdate nolast [] 820 
2 methodius-bio-2 newdate nolast [] 842 
1 metrophanes-bio-6 newdate nolast [] 858 
2 metrophanes-bio-6 newdate nolast [] 869 
3 metrophanes-bio-6 newdate nolast [] 880 
xref fail mettius-curtius-bio-1 CURTIUS METTIUS 1 curtius-mettius-bio-1 5849 1709
148 paren michael-ii-balbus-bio-1 820 False [] at Antioch 821 821
149 paren michael-iii-bio-1 842 False [] the Paulicians 848 848
150 paren michael-iii-bio-1 842 False [] private life 854 854
151 paren michael-iii-bio-1 842 False [] the purpose 866 866
152 paren michael-iii-bio-1 842 False [] same year 866 866
1 michael-bio-10 newdate nolast [] 877 
153 paren mithridates-vi-bio-1 -68 False ['Just', 'App', 'Plut'] following year 66 -66
154 paren mithridat

In [ ]:
print('newdates1',newdates1,'newdates',newdates,'newdates2',newdates2,'sents',sentences)

i = 0
for foo in sorted(entrylens,key=entrylens.get,reverse=True):
    #if(1 and len(entrydates[foo])>0):
   #     continue
    i = i + 1
    if(i>1000):
        continue
    #if(entrylens[foo] < 1000):
    #    continue
    if(len(entrydates[foo]) == 0):
        print(i,entrylens[foo],len(entrydates[foo]),foo,entryheads[foo],entrydates[foo])
        
        
        
        
        

In [4]:
print(len(entrylens))

19902


In [10]:
abbrevs = {}
f = open('smithbio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('([A-Z][a-z][a-z][a-z]\.)',l)):
        m = re.search('([A-Z][a-z][a-z][a-z]\.)',l)
        curabb = m[1]
        if(curabb == 'Hist.'):
            print(l)
        if(curabb in abbrevs):
            abbrevs[curabb] = abbrevs[curabb] + 1
        else:
            abbrevs[curabb] = 1
        l = re.sub('([A-Z][a-z][a-z][a-z]\.)','',l)
    

f.close()

for foo in sorted(abbrevs,key=abbrevs.get,reverse=True):
    print(abbrevs[foo],foo)

<p>2. The illustrious (<foreign xml:lang="grc">Ἰλλούστριος</foreign>), the author of an epigram in the Greek Anthology ( ) " on the quoit of Asclepiades." Nothing more is known of him, unless he be the same person as Ablabius, the Novatian bishop of Nicaea, who was a disciple of the rhetorician Troilus, and himself eminent in the same profession, and who lived under Honorius and Theodosius II., at the end of the fourth and the beginning of the fifth centuries after Christ. (Socrates, <title>Hist. Eccl.</title> 7.12.) </p> 

<p>3. The One-eyed (<foreign xml:lang="grc">ὁ Μονόφθαλμος</foreign>), the pupil and successor in the See of Caesarea of Eusebius <date when-custom="340">A. D. 340</date>, whose life he wrote. (Socrates, <title>Hist. Eccl.</title> 2.4.)

<p>Freind's <title>Hist. of Physic</title>; Sprengel, <title>Hist. de la Méd.</title>; Haller, <title>Biblioth. Medic. Pract.</title>; Barchusen, <title>Hist. Medic.</title></p> 

<p>(<persName xml:lang="grc"><surname full="yes">Ἀδαμ

<p>Nitzsch, <title>Hist. Homeri</title>, i. <ref type="page" n="88">p. 88</ref>; Bernhardy, in his edition of <title>Dionys. Perieg.</title> <ref type="page" n="498">p. 498</ref>, &amp;c., and <hi rend="ital">ad Suidam</hi>, i. <ref type="page" n="1395">p. 1395</ref>; <hi rend="ital">Lobeck, Aglaoph.</hi> ii. <ref type="page" n="990">p. 990</ref>,&amp;c.; Welcker, <hi rend="ital">Der Epische Cyclus</hi>, <ref type="page" n="75">p. 75</ref>, &amp;c.</p> 

<p>For further information respecting Dioscorides and the editions of his work, see Le Clerc, <title>Hist. de la Méd.</title>; Haller, <title>Biblioth. Bolan.</title>; Sprengel, <title>Hist. de la Méd.</title>; Fabric. <title>Biblioth. Graeca</title>; Bostock's <hi rend="ital">History of Medicine</hi>; Choulant, <hi rend="ital">Handbuch der Bücherkunde für die Aeltere Medicin.</hi></p> 

<p> (<abbr>Ath.</abbr> vi. <ref type="page" n="254e">p. 254e</ref>.; and perhaps in  , <foreign xml:lang="grc">Διφίλου</foreign> should be substituted

<p>Cave, <title>Hist. Lit.</title> ad an. 869; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="189">p. 189</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 1440; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="189">p. 189</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 1043; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="195">pp. 195</ref>, <ref type="page" n="196">196</ref>.</p> 

<p>Cave, <title>Hist. Lit.</title> ad an. 878; Fabric. <title>Bibl. Graec.</title> vol. xi. <ref type="page" n="205">p. 205</ref>.</p> 

<p>Vossius, <hi rend="ital">de Hist. Graec.</hi> <ref type="page" n="178">p. 178</ref>, ed. Westermann; Clinton, <hi rend="ital">F. H.</hi> vol. iii. <ref type="page" n="534">p. 534</ref>; Jahn, <hi rend="ital">de Palamede</hi>, <ref type="page" n="31">p. 31</ref>; and more especially Preller, in the <title>Zeitschrift für die Alterthumswissenschaft,</title> 1846, <ref type="page" n="673-688">pp. 673-688</ref>

<p>(<persName xml:lang="grc"><surname full="yes">Θεόδοκος</surname></persName>), the name given by Pococke (in his Latin Version of Abú-l-Faraj, <title>Hist. Dynast.</title> <ref type="page" n="128">p. 128</ref>), and Wüstenfeld (<title>Gesch. der Arab. Aerzte</title>, <ref type="page" n="9">p. 9</ref>) to a Greek physician in the service of Hajáj Ibn Yúsuf, the general of the chalif 'Abdu-l-Malek Ibn Merwán, in the <date when-custom="601-700">seventh century after Christ</date>.

<p>Garnier, <hi rend="ital">Dissertationes</hi>, in vol. 5 of Schulze's edition; Tillemont, <hi rend="ital">Mém.</hi> vol. xiv.; Cave, <title>Hist. Litt. s. a. 423</title>, <ref type="page" n="405">pp. 405</ref>, foll., ed. Basil.; Fabric. <title>Bibl. Graec.</title> vol. vii. <ref type="page" n="429">pp. 429</ref>, foil., vol. viii. <ref type="page" n="277">pp. 277</ref>, foil.; Schulze, <hi rend="ital">De Vita et Scriptis B. Theodoreti Dissertatio</hi>, prefixed to vol. i. of his edition ; Neander, <hi rend

In [15]:
titles = {}
f = open('smithbio-work.xml')

for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)):
        m = re.search('<hi rend="ital">([A-Z][a-z]+\. [^<]+)',l)
        print(l)
        curabb = m[1]
        if(curabb in titles):
            titles[curabb] = titles[curabb] + 1
        else:
            titles[curabb] = 1
        l = re.sub('<hi rend="ital">([A-Z][a-z]+\. [^<]+)','',l)
    

f.close()

for foo in sorted(titles,key=titles.get,reverse=True):
    print(titles[foo],foo)

In [8]:
xrefs = {}
f = open('smithbio-work.xml')

xrefcnt = 0
for l in f:
    if(not re.search('<p>',l)):
        continue
    l = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',l)
    l = re.sub('<title[^>]+>[^<]+</title>',' ',l)
    while(re.search('\[<hi rend="smallcaps">([^<]+)',l)):
        m = re.search('\[<hi rend="smallcaps">([^<]+)',l)
        #print(l)
        curabb = m[1]
        if(curabb in xrefs):
            xrefs[curabb] = xrefs[curabb] + 1
        else:
            xrefs[curabb] = 1
        l = re.sub('\[<hi rend="smallcaps">([^<]+)','',l)
    

f.close()

for foo in sorted(xrefs,key=xrefs.get,reverse=True):
    print(xrefs[foo],foo)

FileNotFoundError: [Errno 2] No such file or directory: 'bio-work.xml'

In [1]:
centuries = {}
    
def cntcents(fname):
    f = open(fname)

    xrefcnt = 0
    for l in f:
        if(not re.search('<p>',l)):
            continue
        l = re.sub('<bibl[^>]+>[<]+</bibl>',' ',l)
        l = re.sub('<title[^>]+>[<]+</title>',' ',l)
        matchs = '([a-zA-Z0-9]+)\s+(century)\s+([AB]\.\s*[DC]\.)'
        while(re.search(matchs,l)):
            m = re.search(matchs,l)
        #print(l)
            curabb = m[1] + ' ' + m[3]
            curabb = re.sub('\.\s+','.',curabb)
            if(curabb in centuries):
                centuries[curabb] = centuries[curabb] + 1
            else:
                centuries[curabb] = 1
            l = re.sub(matchs,'',l)
    

    f.close()
cntcents('smithbio-work.xml')
cntcents('../002/smithgeo-work.xml')

for foo in sorted(centuries):
    if(re.search('B',foo)):
        print('centval["'+foo+'"] = "-100-101"')
    else:
        print('centval["'+foo+'"] = "101-100"')

FileNotFoundError: [Errno 2] No such file or directory: 'bio-work.xml'

In [46]:
def getbestentry(query):
    query = re.sub('^([A-Za-z])\.\s+(.+)','\g<2> \g<1>',query)
    query = re.sub('[\,\.\s]+','-',query)
    queries = query.lower().split('-')
    if(len(queries)> 1):
        filts = '\\b' + queries[1] + '\\b'
    else:
        filts = ''
    if(not queries[0] in singnameindex):
        print(queries[0],'not known')
        return
    workdict = singnameindex[queries[0]]
    retlist = sorted(workdict,key=workdict.get,reverse=True)
    
    #print(retlist[1:5])
    goods = []
    bads = []
    for foo in sorted(workdict,key=workdict.get,reverse=True):
        if(filts and not re.search(filts,foo)):
            #continue
            #print('\tfailfilt',end=' ')
            bads.append(foo)
        else:
            goods.append(foo)
        #print(foo,workdict[foo])
    print('good',goods)
    print('failed',bads)

getbestentry('SCHOLASTICUS')

good ['cometas-scholasticus-bio-1', 'athanasius-scholasticus-bio-1', 'eratosthenes-scholasticus-bio-1', 'arabius-scholasticus-bio-1', 'athanasius-scholasticus-bio-2']
failed []


In [5]:
import re
f = open('smithbio-work.xml')

i = 0
j = 0
k = 0
firstpara = 0
for l in f:
    m = re.search('entry"\s+xml:id="([^"]+-)([0-9abc]+|group|gens|geo)"',l)
    if(m):
        curentryid = m[1]+m[2]
        firstpara = 1
        expectnum = m[2]
        basename = re.sub('(-bio|-group).*$','',curentryid)
    
    
    #if(re.search('<head>',l)):
   #     workl = re.sub('</persName>[^<]+<persName[^>]+>',' ',l)
   # else:
    #    workl = l
    m = re.search('(<head>|<head><label[^>]*>)<persName[^>]*>(.+)</persName>',l)
    if(m):
        l = re.sub('<note[^>]*>.+</note>',' ',l)
        curname = l.lower()
        curname = re.sub('</persname>[^<]+<persName[^>]*>',' ',curname)
        curname = re.sub('<[^>]+>','',curname)
        curname = re.sub("'",'',curname)
        curname = re.sub('[\s\.,]+','-',curname)
        curname = re.sub('-$','',curname)
        if(curentryid == 'cleon-bio-1'):
            print(curname,l,sep='---')
        nameargs = curname.split('-')
        idargs = basename.split('-')
        extraargs = []
        for foo in idargs:
            if(not foo in nameargs):
                extraargs.append(foo)
        #if(not curname == basename):
        if(extraargs):
            k = k + 1
            print('mismatch',k,curentryid,curname,extraargs)
    m = re.search('([A-Z][a-z]+\s+[Gg]ens)',l)
    if(firstpara and re.search('-1$',curentryid) and m):
        curgens = m[1]
        #print(j,'gens',curentryid,curgens,l)
        
        
    m = re.search('<p>([0-9]+)\.',l)
    if(1 and m and firstpara and not m[1] == expectnum):
        i = i + 1
        #print(i,curentryid,l)
        
    if(re.search('<p>',l)):
        firstpara = 0

f.close()

mismatch 1 acherusia-geo-1 acherusia ['geo', '1']
mismatch 2 appuleia-gens-bio-1 appuleia ['gens']
mismatch 3 artavasdes-bio-3 artavasde-or-artabasdus ['artavasdes']
mismatch 4 asclepiades-areius-bio-3 asclepiades ['areius']
mismatch 5 callias-fam-2 callias-and-hipponicus ['fam', '2']
mismatch 6 callias-bio-3 hipponicus-i ['callias']
mismatch 7 callias-bio-5 hipponicus-ii-or-hipponicus-ammon ['callias']
mismatch 8 callias-bio-7 hipponicus-iii ['callias']
mismatch 9 camerinus-gens-bio-1 camerinus ['gens']
mismatch 10 cato-porcius-gens-bio-1 cato-porcius ['gens']
mismatch 11 claudius-gens-bio-1 claudius ['gens']
mismatch 12 claudius-bio-41 p-clodius-pulcher ['claudius']
cleon---<head><persName xml:lang="la"><surname full="yes">Cleon</surname></persName></head> 

mismatch 13 crassus-licinius-bio-11 licinia ['crassus', 'licinius']
mismatch 14 diodorus-tarsensis-bio-17 diodorus-of-tarsus ['tarsensis']
mismatch 15 gregorius-antiochus-bio-12 gregorius-of-antioch ['antiochus']
mismatch 16 jupi

In [4]:
l = '<head><label><persName xml:lang="la"><surname full="yes">Michael</surname> <genName full="yes">I.</genName></persName> or <persName xml:lang="la"><surname full="yes">Michael</surname> <addName full="yes">Rhanga\'be</addName></persName></label></head>'
l = re.sub('<persName[^>]*>(.+)</persName>.*','\g<1>',l)

print(l)


<head><label><surname full="yes">Michael</surname> <genName full="yes">I.</genName></persName> or <persName xml:lang="la"><surname full="yes">Michael</surname> <addName full="yes">Rhanga'be</addName>


In [28]:
f = open('smithbio-work.xml')

firstpara = 0
for l in f:
    m = re.search('entry" xml:id="([^"]+)"',l)
    if(m):
        curentryid = m[1]
        firstpara = 1
    if(re.search('<p>',l) and firstpara):
        firstpara = 0
        if(re.search('(family of|family[\-\s]+name|cognomen|agnomen| surname\\b)',l) and not re.search('(family|gens|cognomen|agnomen|surname)',curentryid)):
            print(curentryid,l)
            

f.close()

abaeus-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Ἀβαῖος</surname></persName>), a surname of Apollo derived from the town of Abae in Phocis, where the god had a rich temple. (Hesych. <hi rend="ital">s. v.</hi> <foreign xml:lang="grc">Ἄβαι</foreign>; <bibl n="Hdt. 8.33">Hdt. 8.33</bibl>; <bibl n="Paus. 10.35.1">Paus. 10.35.1</bibl>, &amp;c.) </p> 

abrettenus-bio-1 <p>(<label xml:lang="grc">Ἀβρεττηνός</label>), a surname of Zeus in Mysia. (<bibl n="Strabo xii.p.574">Strab. xii. <ref type="page" n="574">p. 574</ref></bibl>.) </p> 

acacesius-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Ἀκακήσιος</surname></persName>), a surname of Hermes (Callim. <title>Hym. in Dian.</title> 143), for which Homer (<bibl n="Hom. Il. 16.185">Hom. Il. 16.185</bibl>; <bibl n="Hom. Od. 24.10">Od. 24.10</bibl>) uses the form <foreign xml:lang="grc">ἀκάκητα</foreign> (<foreign xml:lang="grc">ἀκακήτης</foreign>).

acersecomes-bio-1 <p>(<label xml:lang="grc">Ἀκερσεκόμης</label>), a surname o

catus-bio-1 <p>a word indicating shrewdness, caution, sagacity, or the like, was a surname of <abbr>Sex.</abbr> Aelius Paetus, who was consul <date when-custom="-198">B. C. 198</date> [<ref target="paetus-bio-1">PAETUS</ref>], and the cognomen of <abbr>Sex.</abbr> Aelius, consul in <date when-custom="4">A. D. 4</date>, with C. Sentius Saturninus. (<bibl n="Vell. 2.103">Vell. 2.103</bibl>.)</p> 

caudinus-bio-1 <p>a surname of several of the Cornelii Lentuli. [<ref target="lentulus-bio-3">LENTULUS.</ref>] <pb n="657" source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044099890501%25seq=675"/></p> 

causius-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Καούσιος</surname></persName>), a surname of Asclepius, derived from Caus in Arcadia, where he was worshipped. (Steph. <abbr>Byz.</abbr> <hi rend="ital">s. v.</hi> <foreign xml:lang="grc">Καοῦς</foreign>; comp. <bibl n="Paus. 8.25.1">Paus. 8.25.1</bibl>.) </p> 

cenaeus-bio-1 <p>(<persName xml:lang="grc"><surname full="yes">Κηνα

hegesaratus-bio-1 <p>was descended from an ancient and noble family of Larissa in Thessaly, and was leader of the Pompeian party in that city during the civil war in <date when-custom="-48">B. C. 48</date>.

heliodorus-bio-14 <p>The author of the oldest and by far the best of the Greek romances, Heliodorus, the son of Theodosius, was a native of Syria, and was born, not, as Photius says, at Aminda, but at Emesa, as he himself tells us at the <pb n="374" source="https://babel.hathitrust.org/cgi/pt?id=hvd.32044011634888%25seq=400"/> end of his romance :--<foreign xml:lang="grc">Τοιόνδε πέρας ἔσχε τὸ σύνταγμα τῶν περὶ Θεαγένην καὶ Χαρίκλείκλειαν Αἰθιοπικῶν ὁ συνέταξεν ἀνὴρ Φοίνιξ Ἐμεσηνὸς, τῶν ἀφʼ Ἡλίου, γένος, Θεοδοσίον παῖς Ἡλιόδωρος</foreign>. The words <foreign xml:lang="grc">τῶν ἀφʼ Ἡλίου γενος</foreign> no doubt mean that he was of the family of priests of the Syrian god of the Sun (Elagabalus).

hellotia-bio-1 <p>or HELLO'TIS (<foreign xml:lang="grc">Ἐλλωτία</foreign> or <foreign x

nero-bio-1 <p>was a cognomen of the Claudia Gens, which is said to signify, in the Sabine tongue "fortis ac strenuus." (Sueton. <title>Tib. Nero</title>, 1; and the remarks of Gellius, <bibl n="Gel. 13.22">13.22</bibl>.)</p> 

nesiotes-bio-1 <p>a sculptor, appears to have been an assistant of the celebrated Athenian artist Critias, and not a surname of the latter, as some modern writers have conjectured. [<ref target="critias-bio-1">CRITIAS</ref>, <abbr>Vol.</abbr> I. <ref type="page" n="893">p. 893</ref>.]</p> 

nicanor-bio-18 <p>5. Stephanus Byzantinus mentions a writer of this name to whom he adds that of <foreign xml:lang="grc">Λέανδρος</foreign>, as the author of a work called <foreign xml:lang="grc">Μετονοομασιας</foreign>. Athenaeus quotes the same work, but calls the writer a Cyrenian, without giving him the surname.

nice-bio-3 <p>3. Nice also occurs as a surname of Athena, under which the goddess had a sanctuary on the acropolis of Megara. (<bibl n="Paus. 1.42.4">Paus. 1.42.4

scipio-bio-7 <p>6. <persName xml:lang="la"><forename full="yes">Cn.</forename> <surname full="yes">Cornelius</surname> <addName full="yes">Scipio</addName> <addName full="yes">Asina</addName></persName>, the son of No. 5. The reason of his cognomen Asina is related by Macrobius (<bibl n="Macr. 1.6">Macr. 1.6</bibl>).

sciras-bio-2 <p>(<label xml:lang="grc">Σκιράς</label>), a surname of Athena, under which she had a temple in the Attic port of Phaleron, and in the island of Salamis (<bibl n="Paus. 1.1.4">Paus. 1.1.4</bibl>; <bibl n="Hdt. 8.94">Hdt. 8.94</bibl>).

scopas-bio-2 <p>(<persName xml:lang="grc"><surname full="yes">Σκόπας</surname></persName>), one of the most distinguished sculptors of the later Attic school, was a native of Paros, which was then subject to Athens (<bibl n="Strabo xiii.p.604">Strab. xiii. p.604</bibl>; <bibl n="Paus. 8.45.4">Paus. 8.45.4</bibl>); and he appears to have belonged to a family of artists in that island.

seleucus-iii-bio-1 <p>(<persName xml:lang="